In [1]:
!pwd

/home/sagemaker-user/Brain-Tumor-Classification-MRI/notebooks


In [2]:
%cd ..

/home/sagemaker-user/Brain-Tumor-Classification-MRI


# Callbacks Configuration

In [23]:
from dataclasses import dataclass

In [20]:
@dataclass
class CallbacksConfig:
    callbacks_folder: str
    tensorboard_dir: str
    checkpoint_path: str

# Configuration Manager

In [6]:
from src.utils import read_yaml, create_dirs
from src.constants import *
import os

In [24]:
class ConfigManager:
    def __init__(self, config_path=CONFIG_PATH, params_path=PARAMS_PATH):
        self.config = read_yaml(config_path)
        self.artifacts_folder = self.config.artifacts_root
        create_dirs([self.artifacts_folder])

    def get_callbacks_config(self):
        config = self.config.callbacks
        tensorboard_dir = os.path.join(
            self.artifacts_folder, config.folder_name, config.tensorboard_folder_name
        )
        checkpoint_path = os.path.join(
            self.artifacts_folder, config.folder_name, config.checkpoint
        )
        
        create_dirs([
            tensorboard_dir,
            os.path.dirname(checkpoint_path)
        ])

        return CallbacksConfig(
            callbacks_folder = self.config.callbacks,
            tensorboard_dir = tensorboard_dir,
            checkpoint_path = checkpoint_path
        )

# Callbacks Function

In [10]:
import tensorflow as tf
import time

In [27]:
class Callbacks:
    def __init__(self, config: CallbacksConfig):
        self.config = config

    @property
    def _tensorboard(self):
        timestamp = time.strftime("%Y-%m-%d-T%H-%M-%S")
        logs_dir = os.path.join(self.config.tensorboard_dir, timestamp)
        return tf.keras.callbacks.TensorBoard(logs_dir)

    @property
    def _checkpoint(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_path,
            save_best_only = True
        )

    def get_callbacks(self):
        return [self._tensorboard, self._checkpoint]

# Perform The Step

In [28]:
try:
    config = ConfigManager().get_callbacks_config()
    Callbacks(config).get_callbacks()
except Exception as e:
    raise e

[2024-09-26 18:26:25,489: INFO: utils: The file: config.yaml loaded successfully...]
[2024-09-26 18:26:25,492: INFO: utils: The file: params.yaml loaded successfully...]
[2024-09-26 18:26:25,493: INFO: utils: Directory 'artifacts' was successfully created...]
[2024-09-26 18:26:25,495: INFO: utils: Directory 'artifacts/callbacks/tensorboard' was successfully created...]
[2024-09-26 18:26:25,496: INFO: utils: Directory 'artifacts/callbacks/checkpoint' was successfully created...]
